Downloading the VCF file from ClinVar and extracting it.

In [32]:
!wget ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz

--2024-12-11 10:46:41--  ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz
           => ‘clinvar.vcf.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/clinvar/vcf_GRCh38 ... done.
==> SIZE clinvar.vcf.gz ... 104979789
==> PASV ... done.    ==> RETR clinvar.vcf.gz ... done.
Length: 104979789 (100M) (unauthoritative)

clinvar.vcf.gz      100%[===================>] 100.12M  45.6MB/s    in 2.2s    

2024-12-11 10:46:44 (45.6 MB/s) - ‘clinvar.vcf.gz’ saved [104979789]



In [33]:
!gunzip clinvar.vcf.gz

gzip: clinvar.vcf already exists; do you wish to overwrite (y or n)? y


Viewing clinvar.vcf head:

In [34]:
!grep "CLNDN=" clinvar.vcf | head

1	69134	2205837	A	G	.	.	ALLELEID=2193183;CLNDISDB=MedGen:CN169374;CLNDN=not_specified;CLNHGVS=NC_000001.11:g.69134A>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Likely_benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=OR4F5:79501;MC=SO:0001583|missense_variant;ORIGIN=1
1	69314	3205580	T	G	.	.	ALLELEID=3374047;CLNDISDB=MedGen:CN169374;CLNDN=not_specified;CLNHGVS=NC_000001.11:g.69314T>G;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=OR4F5:79501;MC=SO:0001583|missense_variant;ORIGIN=1
1	69423	3205581	G	A	.	.	ALLELEID=3374048;CLNDISDB=MedGen:CN169374;CLNDN=not_specified;CLNHGVS=NC_000001.11:g.69423G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=OR4F5:79501;MC=SO:0001583|missense_variant;ORIGIN=1
1	69581	2252161	C	G	.	.	ALLELEID=2238986;CLNDISDB=MedGen:CN169374;CLNDN=not_specified;CL

Automate Complex Filtering:

In [1]:
# Parse VCF file and filter for specific conditions
!pip install pysam
import pysam
# Save each DataFrame to a CSV file (optional) and print the counts
import os

# Open the VCF file
vcf_file = pysam.VariantFile("clinvar.vcf")
filtered_variants = []
import pandas as pd

# Load the VCF data into a DataFrame (update the path to your filtered VCF file)
vcf_file = "clinvar.vcf"  # Replace with the actual file path
columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df = pd.read_csv(vcf_file, sep="\t", comment='#', names=columns)

# Function to parse INFO field into a dictionary
def parse_info(info):
    parsed = {}
    for entry in info.split(";"):
        if "=" in entry:
            key, value = entry.split("=", 1)
            parsed[key] = value
    return parsed

# Parse the INFO column into separate fields
info_parsed = df["INFO"].apply(parse_info)
info_df = pd.DataFrame(info_parsed.tolist())
df = pd.concat([df, info_df], axis=1)

# Filter for relevant genes and classifications
genes_of_interest = ["EGFR", "KRAS", "ALK"]
classifications = {
    "Pathogenic/Likely Pathogenic": ["Pathogenic", "Likely_pathogenic"],
    "Benign/Likely Benign": ["Benign", "Likely_benign"],
    "VUS": ["Uncertain_significance"]
}

# Create DataFrames for each gene and classification combination
result_dfs = {}
for gene in genes_of_interest:
    for classification_name, classification_values in classifications.items():
        # Filter for the specific gene and classification
        filtered_df = df[
            (df["GENEINFO"].str.contains(gene, na=False)) &
            (df["CLNSIG"].isin(classification_values))
        ]
        # Store the filtered DataFrame in a dictionary
        key = f"{gene}_{classification_name.replace(' ', '_')}"
        result_dfs[key] = filtered_df

# Save each DataFrame to a CSV file
for key, filtered_df in result_dfs.items():
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(f"{key}.csv"), exist_ok=True)
    filtered_df.to_csv(f"{key}.csv", index=False)


<ipython-input-1-10350e4d5771>:15: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(vcf_file, sep="\t", comment='#', names=columns)


In [2]:
for key, filtered_df in result_dfs.items():
    print(f"{key}: {len(filtered_df)} variants")

EGFR_Pathogenic/Likely_Pathogenic: 119 variants
EGFR_Benign/Likely_Benign: 1292 variants
EGFR_VUS: 1409 variants
KRAS_Pathogenic/Likely_Pathogenic: 62 variants
KRAS_Benign/Likely_Benign: 146 variants
KRAS_VUS: 198 variants
ALK_Pathogenic/Likely_Pathogenic: 131 variants
ALK_Benign/Likely_Benign: 2458 variants
ALK_VUS: 3432 variants


In [70]:
df.head()

,Chromosome,Position,ID,Clinical Significance,Disease Name
0,1,11114363,156709,"(Pathogenic,)",(CEBALID_syndrome|Macrocephaly-intellectual_di...
1,1,11124516,156703,"(Pathogenic,)",(not_provided|Isolated_focal_cortical_dysplasi...
2,1,11124516,376129,"(Pathogenic,)",(Papillary_renal_cell_carcinoma_type_1|Renal_c...
3,1,11157173,190121,"(Pathogenic,)",(Glioblastoma|Breast_neoplasm|Macrocephaly-int...
4,1,11157173,376453,"(Pathogenic,)",(Overgrowth_syndrome_and/or_cerebral_malformat...
